In [1]:
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, precision_recall_curve
import json
import matplotlib.pyplot as plt

In [2]:
with open('23_January.json') as json_file:
    data = json.load(json_file)

In [3]:
trial_df = pd.DataFrame(data["values"], columns = ['id', 'observation_id', 'observation', 'proba', 'true_class', 'predicted_class']) 
trial_df.head()

,id,observation_id,observation,proba,true_class,predicted_class
0,7733,7169,"{""id"": 7169, ""observation"": {""Department Name""...",0.473302,NaN,1
1,7734,7170,"{""id"": 7170, ""observation"": {""Department Name""...",0.277561,NaN,0
2,7735,7171,"{""id"": 7171, ""observation"": {""Department Name""...",0.277561,NaN,0
3,7736,7172,"{""id"": 7172, ""observation"": {""Department Name""...",0.233582,NaN,0
4,4,3,"{""id"": 3, ""observation"": {""Department Name"": ""...",0.324282,1.0,0


In [4]:
# Get only observations that received the true_class
trial_df = trial_df[(trial_df.true_class == 1) | (trial_df.true_class == 0)]

In [5]:
precision_score(trial_df.true_class, trial_df.predicted_class)

0.5825575698187163

In [6]:
recall_score(trial_df.true_class, trial_df.predicted_class)

0.6623955431754874

In [7]:
accuracy_score(trial_df.true_class, trial_df.predicted_class)

0.7084

In [8]:
obs_id = trial_df.observation.apply(lambda x: json.loads(x)['id'])
obs = trial_df.observation.apply(lambda x: json.loads(x)['observation']).apply(pd.Series).set_index(obs_id)
obs = obs.reset_index().rename(columns={"observation": "observation_id"})
obs.head()

,observation_id,Department Name,InterventionDateTime,InterventionLocationName,InterventionReasonCode,ReportingOfficerIdentificationID,ResidentIndicator,SearchAuthorizationCode,StatuteReason,SubjectAge,SubjectEthnicityCode,SubjectRaceCode,SubjectSexCode,TownResidentIndicator
0,3,Milford,05/16/2018 04:08:00 AM,MILFORD,E,473,True,I,Other,49.0,N,B,F,False
1,6,Naugatuck,05/16/2018 11:43:00 AM,NAUGATUCK,I,100296,True,I,Administrative Offense,23.0,H,W,M,False
2,7,Naugatuck,05/16/2018 11:43:00 AM,NAUGATUCK,I,100296,True,I,Administrative Offense,23.0,H,W,M,False
3,8,Glastonbury,05/16/2018 12:17:00 PM,Glastonbury,V,BWC0441,True,O,Moving Violation,24.0,N,B,F,False
4,9,Glastonbury,05/16/2018 12:17:00 PM,Glastonbury,V,BWC0441,True,O,Moving Violation,24.0,N,B,F,False


In [9]:
trial_df = trial_df.drop(columns=["observation"])
trial_df = trial_df.merge(obs, on='observation_id')
trial_df.head()

,id,observation_id,proba,true_class,predicted_class,Department Name,InterventionDateTime,InterventionLocationName,InterventionReasonCode,ReportingOfficerIdentificationID,ResidentIndicator,SearchAuthorizationCode,StatuteReason,SubjectAge,SubjectEthnicityCode,SubjectRaceCode,SubjectSexCode,TownResidentIndicator
0,4,3,0.324282,1.0,0,Milford,05/16/2018 04:08:00 AM,MILFORD,E,473,True,I,Other,49.0,N,B,F,False
1,7,6,0.259167,0.0,0,Naugatuck,05/16/2018 11:43:00 AM,NAUGATUCK,I,100296,True,I,Administrative Offense,23.0,H,W,M,False
2,8,7,0.259167,0.0,0,Naugatuck,05/16/2018 11:43:00 AM,NAUGATUCK,I,100296,True,I,Administrative Offense,23.0,H,W,M,False
3,9,8,0.581199,0.0,1,Glastonbury,05/16/2018 12:17:00 PM,Glastonbury,V,BWC0441,True,O,Moving Violation,24.0,N,B,F,False
4,10,9,0.581199,0.0,1,Glastonbury,05/16/2018 12:17:00 PM,Glastonbury,V,BWC0441,True,O,Moving Violation,24.0,N,B,F,False


In [10]:
#get the observations where we got the predicted class wrong
#trial_df = trial_df[trial_df.true_class != trial_df.predicted_class]
#trial_df.head()

In [11]:
precision_female = precision_score(trial_df[trial_df.SubjectSexCode == 'F'].true_class, trial_df[trial_df.SubjectSexCode == 'F'].predicted_class)
precision_male = precision_score(trial_df[trial_df.SubjectSexCode == 'M'].true_class, trial_df[trial_df.SubjectSexCode == 'M'].predicted_class)
precision_hispanic = precision_score(trial_df[trial_df.SubjectEthnicityCode == 'H'].true_class, trial_df[trial_df.SubjectEthnicityCode == 'H'].predicted_class)
precision_middle_eastern = precision_score(trial_df[trial_df.SubjectEthnicityCode == 'M'].true_class, trial_df[trial_df.SubjectEthnicityCode == 'M'].predicted_class)
precision_not_applicable_ethnicity = precision_score(trial_df[trial_df.SubjectEthnicityCode == 'N'].true_class, trial_df[trial_df.SubjectEthnicityCode == 'N'].predicted_class)
precision_white = precision_score(trial_df[trial_df.SubjectRaceCode == 'W'].true_class, trial_df[trial_df.SubjectRaceCode == 'W'].predicted_class)
precision_black = precision_score(trial_df[trial_df.SubjectRaceCode == 'B'].true_class, trial_df[trial_df.SubjectRaceCode == 'B'].predicted_class)
precision_indian = precision_score(trial_df[trial_df.SubjectRaceCode == 'I'].true_class, trial_df[trial_df.SubjectRaceCode == 'I'].predicted_class)
precision_asian = precision_score(trial_df[trial_df.SubjectRaceCode == 'A'].true_class, trial_df[trial_df.SubjectRaceCode == 'A'].predicted_class)
print(precision_female)
print(precision_male)
print(precision_hispanic)
print(precision_middle_eastern)
print(precision_not_applicable_ethnicity)
print(precision_white)
print(precision_black)
print(precision_indian)
print(precision_asian)

0.5751789976133651
0.5844636251541308
0.547085201793722
0.0
0.5924764890282131
0.5994087213599408
0.5505952380952381
0.5
0.5


C:\Anaconda3\envs\capstone_env\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
trial_df.SubjectRaceCode.value_counts()

W    3333
B    1611
A      46
I      10
Name: SubjectRaceCode, dtype: int64